# Population Density Bar Chart

Shows population density (people per sq km) across Toronto neighbourhoods.

## 1. Data Reference

### Source Tables

| Table | Grain | Key Columns |
|-------|-------|-------------|
| `mart_neighbourhood_demographics` | neighbourhood × year | population_density, population, land_area_sqkm |

### SQL Query

In [1]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load .env from project root
load_dotenv("../../.env")

engine = create_engine(os.environ["DATABASE_URL"])

query = """
SELECT
    neighbourhood_name,
    population_density,
    population,
    land_area_sqkm,
    median_household_income,
    pct_renter_occupied
FROM mart_toronto.mart_neighbourhood_demographics
WHERE year = (SELECT MAX(year) FROM mart_toronto.mart_neighbourhood_demographics)
  AND population_density IS NOT NULL
ORDER BY population_density DESC
"""

df = pd.read_sql(query, engine)
print(f"Loaded {len(df)} neighbourhoods")

Loaded 158 neighbourhoods


### Transformation Steps

1. Sort by population density
2. Select top 20 most dense neighbourhoods

In [2]:
data = df.head(20).to_dict("records")

### Sample Output

In [3]:
df[["neighbourhood_name", "population_density", "population", "land_area_sqkm"]].head(
    10
)

,neighbourhood_name,population_density,population,land_area_sqkm
0,North St.James Town,43704.32,18535,0.4241
1,Yonge-Doris,42720.92,20160,0.4719
2,Church-Wellesley,40115.03,22320,0.5564
3,North Toronto,39495.28,15885,0.4022
4,Wellington Place,26091.84,25570,0.9800
5,Harbourfront-CityPlace,24705.83,28135,1.1388
6,Bay-Cloverhill,24131.44,16670,0.6908
7,South Eglinton-Davisville,24065.84,22735,0.9447
8,Downtown Yonge East,21884.27,17700,0.8088
9,Regent Park,19600.31,12750,0.6505


## 2. Data Visualization

### Figure Factory

Uses `create_horizontal_bar` from `portfolio_app.figures.toronto.bar_charts`.

In [4]:
import sys

sys.path.insert(0, "../..")

from portfolio_app.figures.toronto.bar_charts import create_horizontal_bar

fig = create_horizontal_bar(
    data=data,
    name_column="neighbourhood_name",
    value_column="population_density",
    title="Top 20 Most Dense Neighbourhoods",
    color="#9C27B0",
    value_format=",.0f",
)

fig.show()

### Density Statistics

In [5]:
print("City-wide Statistics:")
print(f"  Total Population: {df['population'].sum():,.0f}")
print(f"  Total Area: {df['land_area_sqkm'].sum():,.1f} sq km")
print(f"  Average Density: {df['population_density'].mean():,.0f} per sq km")
print(f"  Max Density: {df['population_density'].max():,.0f} per sq km")
print(f"  Min Density: {df['population_density'].min():,.0f} per sq km")

City-wide Statistics:
  Total Population: 2,761,290
  Total Area: 642.8 sq km
  Average Density: 7,142 per sq km
  Max Density: 43,704 per sq km
  Min Density: 861 per sq km
